In [69]:
import mlx.core as mx
import mlx.nn as nn
import mlx.optimizers as optim

import numpy as np

In [70]:
class MLP(nn.Module):
    def __init__(
        self, num_layers: int, input_dim: int, hidden_dim: int, output_dim: int
    ):
        super().__init__()
        layer_sizes = [input_dim] + [hidden_dim] * num_layers + [output_dim]
        self.layers = [
            nn.Linear(idim, odim)
            for idim, odim in zip(layer_sizes[:-1], layer_sizes[1:])
        ]

    def __call__(self, x):
        for l in self.layers[:-1]:
            x = mx.maximum(l(x), 0.0)
        return self.layers[-1](x)

## Different types of loss functions we can use:

In [71]:
def loss_fn(model, X, y):
    return mx.mean(nn.losses.cross_entropy(model(X), y))

In [72]:
def mse_loss_fn(model, X, y):
    return mx.mean(nn.losses.mse_loss(model(X), y))

In [73]:
def eval_fn(model, X, y):
    return mx.mean(mx.argmax(model(X), axis=1) == y)

In [74]:
num_layers = 2
hidden_dim = 32
num_classes = 10
batch_size = 256
num_epochs = 10
learning_rate = 1e-1

# Load the data
import mnist
train_images, train_labels, test_images, test_labels = map(
    mx.array, mnist.mnist()
)

In [75]:
def batch_iterate(batch_size, X, y):
    perm = mx.array(np.random.permutation(y.size))
    for s in range(0, y.size, batch_size):
        ids = perm[s : s + batch_size]
        yield X[ids], y[ids]

In [76]:
# Load the model
model = MLP(num_layers, train_images.shape[-1], hidden_dim, num_classes)

print(model)

MLP(
  (layers.0): Linear(input_dims=784, output_dims=32, bias=True)
  (layers.1): Linear(input_dims=32, output_dims=32, bias=True)
  (layers.2): Linear(input_dims=32, output_dims=10, bias=True)
)


In [77]:
model.parameters()

{'layers': [{'weight': array([[-0.0348091, -0.00404139, -0.00787482, ..., -0.0158604, 0.0113245, -0.0224375],
          [0.0267361, 0.0260765, -0.00518377, ..., 0.0159355, -0.0146609, 0.005239],
          [0.0303898, -0.010741, -0.0292016, ..., 0.0198094, -0.0211013, -0.00996849],
          ...,
          [-0.0318463, -0.0197012, -0.0287083, ..., 0.00663459, 0.0249421, 0.0167761],
          [-0.0319448, -0.0346811, 0.00882965, ..., 0.010089, 0.00793833, 0.0319544],
          [0.00984352, 0.0345296, 0.0198374, ..., -0.0208674, -0.000505667, 0.00596671]], dtype=float32),
   'bias': array([-0.0108264, 0.00637251, 0.0292783, ..., 0.0170724, 0.0151225, 0.0343075], dtype=float32)},
  {'weight': array([[-0.0585833, 0.0382155, -0.144778, ..., -0.0819081, -0.171623, -0.150085],
          [-0.0690722, -0.0673616, 0.0676869, ..., -0.00347362, -0.125046, -0.149613],
          [0.0834324, 0.0604245, -0.0918727, ..., 0.124461, -0.0188333, 0.0923586],
          ...,
          [-0.148513, -0.135197, 0

In [78]:
mx.eval(model.parameters())

In [88]:
# Get a function which gives the loss and gradient of the
# loss with respect to the model's trainable parameters
loss_and_grad_fn = nn.value_and_grad(model, loss_fn)  # Create a function that returns the loss and its gradients

# Instantiate the optimizer
optimizer = optim.SGD(learning_rate=learning_rate)  # Initialize the optimizer with a specific learning rate

for e in range(num_epochs):  # Outer loop for iterating through each epoch
    for X, y in batch_iterate(batch_size, train_images, train_labels):  # Inner loop for iterating through each mini-batch
        loss, grads = loss_and_grad_fn(model, X, y)  # Compute the loss and gradients for the current batch
        
        # Update the optimizer state and model parameters
        # in a single call
        optimizer.update(model, grads)  # Update model parameters using the computed gradients
        
        # Force a graph evaluation
        mx.eval(model.parameters(), optimizer.state)  # Ensure all computations are evaluated and parameters are updated

    accuracy = eval_fn(model, test_images, test_labels)  # Evaluate model accuracy on the test dataset after each epoch
    print(f"Epoch {e}: Test accuracy {accuracy.item():.3f}")  # Print the epoch number and the corresponding test accuracy

Epoch 0: Test accuracy 0.972
Epoch 1: Test accuracy 0.903
Epoch 2: Test accuracy 0.972
Epoch 3: Test accuracy 0.973
Epoch 4: Test accuracy 0.972
Epoch 5: Test accuracy 0.971
Epoch 6: Test accuracy 0.971
Epoch 7: Test accuracy 0.973
Epoch 8: Test accuracy 0.974
Epoch 9: Test accuracy 0.972
